In [ ]:
import sys
%load_ext autoreload
%autoreload 2

import xfmkit.entry_processed as entry_processed
import xfmkit.clustering as clustering

#add the parent folder to sys path so we can import from within the notebook subfolder
#sys.path.insert(0,'..')

print(sys.path)

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
#args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full"]
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/nf_demo_short"]

PACKAGE_CONFIG='xfmkit/config.yaml'

In [ ]:
pxs, embedding, categories, classavg, palette, kde = entry_processed.read_processed(args_in)

In [ ]:
pixels = pxs.data.d

pixels.shape


In [ ]:
from minisom import MiniSom

import numpy as np

In [ ]:
#neuron map size
m, n = 5, 5

# SOM initialization and training
print('training...')
som = MiniSom(m, n, pixels.shape[1], sigma=1.,
              learning_rate=0.1, neighborhood_function='bubble')  # 3x3 = 9 final colors
som.random_weights_init(pixels)
starting_weights = som.get_weights().copy()  # saving the starting weights
som.train(pixels, 20000, random_order=True, verbose=True)

In [ ]:
print('quantization...')
qnt = som.quantization(pixels)  # quantize each pixels of the image

final_weights = som.get_weights()


In [ ]:
winners = som.win_map(pixels, return_indices=True)

In [ ]:
for i in range(m):
    for j in range(n):
        print(f"({i},{j})")
        print(len(winners[(i,j)]))

In [ ]:
"""
COLOURS
"""

In [ ]:
import seaborn as sns
import colorcet as cc
import numpy as np
import matplotlib.pyplot as plt

cc_palette=sns.color_palette(cc.glasbey_light,100)


cc_palette

In [ ]:
coloursom = MiniSom(m, n, len(cc_palette[0]), sigma=1.,
              learning_rate=0.2, neighborhood_function='gaussian')

coloursom.train(cc_palette, 10000, random_order=True, verbose=True)

winners = coloursom.win_map(cc_palette, return_indices=False)


In [ ]:
neuron_colours=np.zeros((m,n,3), dtype=np.float32)

for i in range(m):
    for j in range(n):
        print(f"({i},{j})")
        print(len(winners[(i,j)]))
        neuron_colours[i,j,:]=winners[(i,j)][0]

plt.imshow(abs(neuron_colours), interpolation='none')

#colours from weights - should be similar
#plt.imshow(abs(coloursom.get_weights()), interpolation='none')

In [ ]:
"""
use winner to get winning neuron for every pixel

assign colour based on colour quantization

"""

In [ ]:
#flatten neuron indices
som_categories=np.zeros(pixels.shape[0], dtype=np.int32)

for i in range(pixels.shape[0]):
    _h, _k = som.winner(pixels[i])
    som_categories[i]=_h*m+_k

som_categories

In [ ]:
#create a new palette from som colours
import copy

nc = np.reshape(neuron_colours, (m*n,-1))

som_palette = copy.deepcopy(cc_palette)

del som_palette[0:]

for i in range(nc.shape[0]):
    som_palette.append(nc[i])

som_palette

In [ ]:
import xfmkit.visualisations as vis

vis.category_map(som_categories, pxs.dimensions, palette=som_palette)

In [ ]:
vis.category_map(categories, pxs.dimensions, palette=palette)